In [32]:
%load_ext autoreload
%autoreload 2

import sys
import pandas as pd
#ADD YOUR DIRECTORY HERE
sys.path.append('/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/src/')
sys.path.append('/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/src/utils')
sys.path.append('/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/src/models')

from utils.analysis import *
from utils.getters import * 
from utils.plotters import * 
from utils.helpers import * 
from utils.config import * 
from models_config.model_config_GSP import *


import matplotlib.pyplot as plt

plt.rcParams['text.latex.preamble']=[r"\usepackage{lmodern}"]
params = {
    'text.usetex' : True,
    'font.size' : 11,
    'font.family' :'lmodern'
}
plt.rcParams.update(params)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/var/folders/1j/5s36wz6s0jlb1k64vsq1k2zh0000gn/T/ipykernel_74385/1891306872.py:21: MatplotlibDeprecationWarning: Support for setting an rcParam that expects a str value to a non-str value is deprecated since 3.5 and support will be removed two minor releases later.
  plt.rcParams['text.latex.preamble']=[r"\usepackage{lmodern}"]


## Model Size 

In [47]:
model_1_dir = '/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/model_data/gender_data/gcn/model_assessment/gcn_student/models/gcn_student_MainModel_3Fold_gender_data_gcn_student_run_0_fixed_init_CV_0_view_0.pt'
model_2_dir = '/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/model_data/gender_data/gcn/model_assessment/gcn/models/gcn_MainModel_3Fold_gender_data_gcn_run_0_fixed_init_CV_0_view_0.pt'
model_3_dir = '/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/model_data/gender_data/gcn/model_assessment/gcn/models/gcn_MainModel_3Fold_gender_data_gcn_run_0_fixed_init_layers_3_CV_0_view_0.pt'

gcn_student = torch.load(model_1_dir)
gcn_teacher = torch.load(model_2_dir)

In [48]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(gcn_student), count_parameters(gcn_teacher)

(108, 2470)

In [49]:
def percentage_decrease(original_value, new_value):
    decrease = original_value - new_value
    percentage = (decrease / original_value) * 100
    return percentage

percentage_decrease(2470, 108)

95.62753036437248

In [50]:
3.5 * 1.18

4.13

In [51]:
percentage_decrease(6.36, 7.76e-1)

87.79874213836479

In [52]:
model_1_dir = '/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/model_data/gender_data/gat/model_assessment/gat_student/models/gat_student_MainModel_3Fold_gender_data_gat_student_run_0_fixed_init_CV_0_view_0_with_teacher.pt'
model_2_dir = '/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/model_data/gender_data/gat/model_assessment/gat/models/gat_MainModel_3Fold_gender_data_gat_run_0_fixed_init_CV_0_view_0.pt'
gat_student = torch.load(model_1_dir)
gat_teacher = torch.load(model_2_dir)


In [53]:
count_parameters(gat_student), count_parameters(gat_teacher)

(110, 2536)

In [54]:
percentage_decrease(2536, 110)

95.66246056782335

## Inference time 

In [41]:
import time
from torch.autograd import Variable

def inference_time(model, dataset="gender_data"):
    
    model.eval()
    
    model_args= gcn_student_lsp_ensamble_2_args
    
    
    G_list = load_data(dataset, 0, NormalizeInputGraphs=False, SAVE_DIR_DATA=SAVE_DIR_DATA)

    folds = stratify_splits(G_list, 10)
        
    [random.shuffle(folds[i]) for i in range(len(folds))]
    train_set, validation_set, test_set = datasets_splits(folds, model_args, 0)
    train_dataset, val_dataset, threshold_value = model_assessment_split(train_set, validation_set, test_set, gcn_student_args)

    for batch_idx, data in enumerate(train_dataset):

        adj = Variable(data['adj'].float(), requires_grad=False).to(device)
        adj = torch.squeeze(adj)

        features = np.identity(adj.shape[0])
        features = Variable(torch.from_numpy(features).float(), requires_grad=False).to("cpu")
        if gcn_student_args["threshold"] in ["median", "mean"]:
            adj = torch.where(adj > threshold_value, torch.tensor([1.0]).to("cpu"), torch.tensor([0.0]).to("cpu"))

        begin_time = time.time()

        model(features, adj)

        return time.time() - begin_time


In [45]:
inference_time_1 = [inference_time(gcn_student) for i in range(100)]
inference_time_2 = [inference_time(gcn_teacher) for i in range(100)]

Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:

In [46]:
np.mean(inference_time_1), np.mean(inference_time_2)

(0.0002826142311096191, 0.00033363103866577146)

In [59]:
inference_time_1 = [inference_time(gat_student) for i in range(100)]
inference_time_2 = [inference_time(gat_teacher) for i in range(100)]

Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:  629 ; Num test graphs:  69
Num training graphs:

In [60]:
np.mean(inference_time_1), np.mean(inference_time_2)

(0.0007356238365173339, 0.005142099857330322)

# BREASTMNIST

In [61]:
model_1_dir = '/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/model_data/BreastMNIST/gcn/model_assessment/gcn_student/models/gcn_student_MainModel_3Fold_BreastMNIST_gcn_student_run_0_fixed_init_CV_0_view_-1_with_teacher.pt'
model_2_dir = '/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/model_data/BreastMNIST/gcn/model_assessment/gcn/models/gcn_MainModel_3Fold_BreastMNIST_gcn_run_0_fixed_init_CV_0_view_-1.pt'

gcn_student = torch.load(model_1_dir)
gcn_teacher = torch.load(model_2_dir)

In [62]:
model_1_dir = '/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/model_data/BreastMNIST/gat/model_assessment/gat/models/gat_MainModel_3Fold_BreastMNIST_gat_run_0_fixed_init_CV_0_view_-1.pt'
model_2_dir = '/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/model_data/BreastMNIST/gat/model_assessment/gat_student/models/gat_student_MainModel_3Fold_BreastMNIST_gat_student_run_0_fixed_init_CV_0_view_-1_with_teacher.pt'

gat_student = torch.load(model_2_dir)
gat_teacher = torch.load(model_1_dir)

In [63]:
inference_time_1 = [inference_time(gcn_student, "BreastMNIST") for i in range(100)]
inference_time_2 = [inference_time(gcn_teacher, "BreastMNIST") for i in range(100)]

Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:

In [64]:
np.mean(inference_time_1), np.mean(inference_time_2)

(0.000276033878326416, 0.00034616231918334963)

In [65]:
inference_time_1 = [inference_time(gat_student, "BreastMNIST") for i in range(100)]
inference_time_2 = [inference_time(gat_teacher, "BreastMNIST") for i in range(100)]

Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:  702 ; Num test graphs:  78
Num training graphs:

In [66]:
np.mean(inference_time_1), np.mean(inference_time_2)

(0.000699605941772461, 0.005078885555267334)